In [ ]:
%matplotlib inline

from __future__ import print_function, division

import os, sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
from ipywidgets import RadioButtons, fixed 

import seaborn as sns
sns.set(context="poster", font_scale=1.5)


## import from local files

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))

from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   gamma, E_0, metallicity_solar
from clustered_SNe.analysis.sedov.dimensionalize_sedov import dimensionalized_sedov
from clustered_SNe.analysis.sedov.closed_form_sedov import SedovSolution
from clustered_SNe.analysis.parse import RunSummary, Overview, cols
from clustered_SNe.analysis.visualize_helpers import single_run, \
                                                     conduction_comparisons, \
                                                     SNe_distplot, \
                                                     plot_zones, \
                                                     plot_shock_location, \
                                                     plot_energy, \
                                                     plot_momentum, \
                                                     plot_luminosity

        
# Holds the most recent run, if more analysis is desired
run_summary = RunSummary()    

In [ ]:
data_dir = "../src/"
data_dir = data_dir_default


# you don't need the full uuid, just enough to distinguish it
id_default = ""

result_widget = interact_manual(single_run,
                                data_dir = widgets.Text(data_dir),
                                id = widgets.Text(id_default))
def update_run_summary_single_run(widget):
    """Copy the latest result into run_summary
    
    Don't reuse this function definition; needs to be bound to the correct result_widget"""
    run_summary.replace_with(result_widget.widget.result)
result_widget.widget.children[0].on_submit(update_run_summary_single_run)
result_widget.widget.children[1].on_submit(update_run_summary_single_run)
result_widget.widget.children[2].on_click( update_run_summary_single_run)



In [ ]:
print(run_summary.first_unreasonable_energy())
print(run_summary.is_time_resolved())
print(run_summary.is_converged())

In [ ]:
np.argmax(run_summary.momentum)

In [ ]:
len(run_summary.momentum)

In [ ]:
plot_zones(run_summary)

In [ ]:
plot_shock_location(run_summary)

In [ ]:
_ = interact(plot_energy,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))

In [ ]:
_ = interact(plot_momentum,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]),
         clear_previous=fixed(True),
         distplot=fixed(True),
         y_axis_scaling=fixed("SNe"),
         plot_times_in_Myr=fixed(True),
         with_momentum_axvline=fixed(True),
         label=fixed(""))

In [ ]:
_ = interact(plot_luminosity,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))
